<a href="https://colab.research.google.com/github/claudiodanielpc/ferrocarriles/blob/main/comercio_aduana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Borrar datos del entorno
rm(list=ls())


#Se utiliza pacman para instalar y cargar paquetes
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,janitor)

Loading required package: pacman

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pacman’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’



janitor installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’, ‘RcppArmadillo’, ‘survey’



srvyr installed



In [2]:
# -------------------------------
# Datos de comercio exterior
# -------------------------------
url_vivi <- "https://www.inegi.org.mx/contenidos/programas/comext/datosabiertos/conjunto_de_datos_bcmm_mensual_mtra_csv.zip"

In [3]:
# -------------------------------
# Descargar ZIP
# -------------------------------
zip_path <- "conjunto_de_datos_bcmm_mensual_mtra_csv.zip"
download.file(
  url_vivi,
  destfile = zip_path,
  mode = "wb"
)
# -------------------------------
# Crear carpeta destino
# -------------------------------
output_dir <- "conjunto_de_datos_bcmm_mensual_mtra_csv"
dir.create(output_dir, showWarnings = FALSE)


In [4]:
# -------------------------------
# Descomprimir
# -------------------------------
unzip(zip_path, exdir = output_dir)

# Ver archivos
list.files(output_dir)


[1] "catalogos"               "conjunto_de_datos"      
[3] "diccionario_de_datos"    "metadatos"              
[5] "modelo_entidad_relacion"

In [52]:
## Leer comercio
comercio <- read_csv(paste0(output_dir,"/conjunto_de_datos/bcmm_mtra_capitulo_mensual_tr_cifra_2012_2025.csv"))%>%
  clean_names()%>% # quitar agregadores en "concepto"
  filter(!concepto %in% c("Marítimo", "Aéreo", "Ferroviario", "Carretero", "Otros")) %>%
  # quitar NA en mtra
  filter(!is.na(mtra)) %>%
  # parsear mtra
  mutate(
    transporte = case_when(
      mtra == "01" ~ "Marítimo",
      mtra == "04" ~ "Aéreo",
      mtra == "06" ~ "Ferroviario",
      mtra == "07" ~ "Carretero",
      mtra == "09" ~ "Otros",
      TRUE ~ mtra
    ),
#Extraer capítulo
 capitulo = str_extract(concepto, "(?<=Capítulo\\s)\\d{2}") %>% as.integer(),
 #Dejar solo capítulo 10 Cereales
  tipo_comercio = case_when(
    capitulo ==10 ~ "Primario",
    TRUE ~ "Otros"
  )
)

head(comercio,5)

Rows: 163994 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): PROD_EST, COBERTURA, TIPO, MES, CONCEPTO, MTRA, ESTATUS_CIFRA, ESTATUS
dbl (2): ANIO, VAL_USD

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


prod_est,cobertura,tipo,anio,mes,concepto,mtra,val_usd,estatus_cifra,estatus,transporte,capitulo,tipo_comercio
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>
BCMM. Modo de transporte por capítulo,Nacional,Exportación,2012,01,Capítulo 01 Animales vivos,04,0.025641,Disponible,Cifras Definitivas.,Aéreo,1,Otros
BCMM. Modo de transporte por capítulo,Nacional,Exportación,2012,01,Capítulo 02 Carne y despojos comestibles,04,0.761302,Disponible,Cifras Definitivas.,Aéreo,2,Otros
BCMM. Modo de transporte por capítulo,Nacional,Exportación,2012,01,"Capítulo 03 Pescados, crustáceos y moluscos",04,8.044272,Disponible,Cifras Definitivas.,Aéreo,3,Otros
BCMM. Modo de transporte por capítulo,Nacional,Exportación,2012,01,"Capítulo 04 Leche, lácteos, huevos y miel",04,0.160430,Disponible,Cifras Definitivas.,Aéreo,4,Otros
BCMM. Modo de transporte por capítulo,Nacional,Exportación,2012,01,Capítulo 05 Otros productos de origen animal,04,0.028720,Disponible,Cifras Definitivas.,Aéreo,5,Otros


In [55]:
unique(comercio$tipo_comercio)

[1] "Otros"    "Primario"

In [56]:
comercio %>%
  # quedarnos solo con sector primario
  filter(tipo_comercio == "Primario") %>%
    #Filtro modo terrestre
  filter(transporte=="Carretero" | transporte=="Ferroviario")%>%
  # sumar valor por año, tipo e transporte
  group_by(anio, tipo, transporte) %>%
  summarise(
    valor_usd = sum(val_usd, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  # calcular total anual por tipo
  group_by(anio, tipo) %>%
  mutate(
    total_anual = sum(valor_usd),
    porcentaje = valor_usd / total_anual * 100
  ) %>%
  ungroup()%>%
  #Solo 2024
  filter(anio==2024)

anio,tipo,transporte,valor_usd,total_anual,porcentaje
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
2024,Exportación,Carretero,36.345,47.563,76.414440
2024,Exportación,Ferroviario,11.218,47.563,23.585560
2024,Importación,Carretero,316.651,4656.943,6.799546
2024,Importación,Ferroviario,4340.292,4656.943,93.200454
